In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
from datasets import load_dataset

In [ ]:
dataset_name = "sahil2801/CodeAlpaca-20k"

In [ ]:
dataset_name

'sahil2801/CodeAlpaca-20k'

In [ ]:
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
len(dataset)

20022

In [ ]:
print(dataset.column_names)

['instruction', 'input', 'output']


In [ ]:
dataset[0]['input']

''

In [ ]:
type(dataset)

datasets.arrow_dataset.Dataset

In [ ]:
dataset.to_csv("test.csv")

Creating CSV from Arrow format:   0%|          | 0/21 [00:00<?, ?ba/s]

6061602

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/test.csv')

In [ ]:
df.head()

,instruction,input,output
0,Create an array of length 5 which contains all...,NaN,"arr = [2, 4, 6, 8, 10]"
1,Formulate an equation to calculate the height ...,NaN,Height of triangle = opposite side length * si...
2,Write a replace method for a string class whic...,"string = ""Hello World!""\nreplace_with = ""Greet...","def replace(self, replace_with):\n new_stri..."
3,Create an array of length 15 containing number...,NaN,"arr = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33..."
4,Write a function to find the number of distinc...,"matrix = [[1, 0, 0],\n [1, 0, 1],\n ...",def find_num_distinct_states(matrix):\n sta...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20022 entries, 0 to 20021
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  20022 non-null  object
 1   input        10257 non-null  object
 2   output       20016 non-null  object
dtypes: object(3)
memory usage: 469.4+ KB


In [ ]:
df.isnull().sum()

instruction       0
input          9765
output            6
dtype: int64

In [ ]:
df = df.fillna(value='None')

In [ ]:
df.isnull().sum()

instruction    0
input          0
output         0
dtype: int64

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model name
new_model = "llama-2-7b-code"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
df.head()

,instruction,input,output
0,Create an array of length 5 which contains all...,None,"arr = [2, 4, 6, 8, 10]"
1,Formulate an equation to calculate the height ...,None,Height of triangle = opposite side length * si...
2,Write a replace method for a string class whic...,"string = ""Hello World!""\nreplace_with = ""Greet...","def replace(self, replace_with):\n new_stri..."
3,Create an array of length 15 containing number...,None,"arr = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33..."
4,Write a function to find the number of distinc...,"matrix = [[1, 0, 0],\n [1, 0, 1],\n ...",def find_num_distinct_states(matrix):\n sta...


In [ ]:
def concatenate_row_with_column_names(row):
    return ' '.join([f"{col} {row[col]}" for col in df.columns])

# Step 3: Apply the function to each row
df['combined'] = df.apply(concatenate_row_with_column_names, axis=1)

In [ ]:
df[['combined']].to_csv('new.csv', index=False)

In [ ]:
pip install pandas datasets

In [ ]:
import pandas as pd
from datasets import Dataset

In [ ]:
df=pd.read_csv('/content/new.csv')

In [ ]:
df.head()

,combined
0,instruction Create an array of length 5 which ...
1,instruction Formulate an equation to calculate...
2,instruction Write a replace method for a strin...
3,instruction Create an array of length 15 conta...
4,instruction Write a function to find the numbe...


In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['combined'],
    num_rows: 20022
})

In [ ]:
dataset.save_to_disk('/Users/abhinavnagaboyina/Downloads/code')

Saving the dataset (0/1 shards):   0%|          | 0/20022 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk
dataset1 = load_from_disk('/Users/abhinavnagaboyina/Downloads/code')


In [ ]:
dataset1

Dataset({
    features: ['combined'],
    num_rows: 20022
})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20022 entries, 0 to 20021
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   combined  20022 non-null  object
dtypes: object(1)
memory usage: 156.5+ KB


In [ ]:
df.to_csv('data.csv', index=False)

In [ ]:
dataset = load_dataset('csv', data_files=['data.csv'], split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field='combined',
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/20022 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.429700
50,2.320500
75,1.077500
100,1.514200
125,0.903500
150,1.149000
175,0.836200
200,1.055000
225,0.848800
250,1.148000


Step,Training Loss
25,1.429700
50,2.320500
75,1.077500
100,1.514200
125,0.903500
150,1.149000
175,0.836200
200,1.055000
225,0.848800
250,1.148000


In [ ]:
print(dataset.column_names)


['combined']


In [ ]:
#  %load_ext tensorboard
#  %tensorboard --logdir results/runs

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Create an array of length 5 which contains all even numbers between 1 and 10."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] Create an array of length 5 which contains all even numbers between 1 and 10. [/INST]  Sure! Here is an array of length 5 which contains all even numbers between 1 and 10:

[2, 4, 6, 8, 10]


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Create a program that finds all the prime numbers less than or equal to a given number."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Create a program that finds all the prime numbers less than or equal to a given number. [/INST]  Sure! Here is a program that finds all the prime numbers less than or equal to a given number:

def find_primes(n):
    primes = []
    for i in range(2, n+1):
        if is_prime(i):
            primes.append(i)
    return primes

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

if __name__ == '__main__':
    n = 10
    primes = find_primes(n)
    print(primes)

Output:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

This program uses a loop to iterate over the numbers from 2 to n and checks if the number is a prime. If it is, it is added to the list of prime numbers. The program then returns the list of prime numbers.

Note: The function is_prime() is used to check if a number is a prime.
